In [2]:
from pyscenic.rnkdb import FeatherRankingDatabase, RankingDatabase
from pyscenic.genesig import GeneSignature
from typing import Type, Tuple
import os
import numpy as np
import pandas as pd
from feather.api import write_dataframe, FeatherReader
from tqdm import tqdm

In [3]:
DB_FOLDER = "/Users/bramvandesande/Projects/lcb/databases"

In [4]:
db = FeatherRankingDatabase(fname=os.path.join(DB_FOLDER, "hg19-regions-220330-9species.extracted.feather"),
                            name="regions", nomenclature="HGNC")

In [5]:
len(db.genes)

220329

In [6]:
IDENTIFIERS_FNAME_EXTENSION = "identifiers.txt"
INVERTED_DB_DTYPE = np.uint32
INDEX_NAME = "features"

In [7]:
def _derive_identifiers_fname(fname):
    return '{}.{}'.format(os.path.splitext(fname)[0], IDENTIFIERS_FNAME_EXTENSION)

In [8]:
def invert(db: Type[RankingDatabase], fname: str, top_n_identifiers: int = 50000) -> None:
    """
    Create an inverted whole genome rankings database keeping only the top n genes/regions for a feature.

    Inverted design: not storing the rankings for all regions in the dataframe but instead store the identifier of the
    top n genes/regions in the dataframe introduces an enormous reduction in disk and memory size.

    :param db: The rankings database.
    :param fname: the filename of the inverted database to be created.
    :param top_n_identifiers: The number of genes to keep in the inverted database.
    """

    df_original = db.load_full()
    n_features = len(df_original)

    index_fname = _derive_identifiers_fname(fname)
    assert not os.path.exists(index_fname), "Database index {0:s} already exists.".format(index_fname)
    identifiers = df_original.columns.values
    with open(index_fname, 'w') as f:
        f.write('\n'.join(identifiers))
    identifier2idx = {identifier: idx for idx, identifier in enumerate(identifiers)}

    inverted_data = np.empty(shape=(n_features, top_n_identifiers), dtype=INVERTED_DB_DTYPE)
    df_original.columns = [identifier2idx[identifier] for identifier in df_original.columns]
    for idx, (_, row) in tqdm(enumerate(df_original.iterrows())):
        inverted_data[idx, :] = np.array(row.sort_values(ascending=True).head(top_n_identifiers).index, dtype=INVERTED_DB_DTYPE)
    df = pd.DataFrame(data=inverted_data, index=df_original.index, columns=list(range(top_n_identifiers)))

    df.index.name = INDEX_NAME
    df.reset_index(inplace=True) # Index is not stored in feather format. https://github.com/wesm/feather/issues/200
    write_dataframe(df, fname)

In [88]:
#invert(db, os.path.join(DB_FOLDER, "hg19-regions-220330-9species.inverted.feather"))

9713it [09:27, 17.13it/s]


In [9]:
from cytoolz import memoize
from numba import njit, prange, uint32, jit

In [12]:
class InvertedRankingDatabase(RankingDatabase):
    @classmethod
    def _derive_identifiers_fname(cls, fname):
        return '{}.{}'.format(os.path.splitext(fname)[0], IDENTIFIERS_FNAME_EXTENSION)


    @classmethod
    def invert(cls, db: Type[RankingDatabase], fname: str, top_n_identifiers: int = 50000) -> None:
        """
        Create an inverted whole genome rankings database keeping only the top n genes/regions for a feature.
        Inverted design: not storing the rankings for all regions in the dataframe but instead store the identifier of the
        top n genes/regions in the dataframe introduces an enormous reduction in disk and memory size.
        :param db: The rankings database.
        :param fname: the filename of the inverted database to be created.
        :param top_n_identifiers: The number of genes to keep in the inverted database.
        """

        df_original = db.load_full()
        n_features = len(df_original)

        index_fname = InvertedRankingDatabase._derive_identifiers_fname(fname)
        assert not os.path.exists(index_fname), "Database index {0:s} already exists.".format(index_fname)
        identifiers = df_original.columns.values
        with open(index_fname, 'w') as f:
            f.write('\n'.join(identifiers))
        identifier2idx = {identifier: idx for idx, identifier in enumerate(identifiers)}

        inverted_data = np.empty(shape=(n_features, top_n_identifiers), dtype=INVERTED_DB_DTYPE)
        df_original.columns = [identifier2idx[identifier] for identifier in df_original.columns]
        for idx, (_, row) in tqdm(enumerate(df_original.iterrows())):
            inverted_data[idx, :] = np.array(row.sort_values(ascending=True).head(top_n_identifiers).index, dtype=INVERTED_DB_DTYPE)
        df = pd.DataFrame(data=inverted_data, index=df_original.index, columns=list(range(top_n_identifiers)))

        df.index.name = INDEX_NAME
        df.reset_index(inplace=True) # Index is not stored in feather format. https://github.com/wesm/feather/issues/200
        write_dataframe(df, fname)


    def __init__(self, fname: str, name: str = None, nomenclature: str = None):
        """
        Create a new inverted database.
        :param fname: The filename of the database.
        :param name: The name of the database.
        :param nomenclature: The nomenclature used for the genes that are being ranked.
        """
        super().__init__(name=name, nomenclature=nomenclature)

        assert os.path.isfile(fname), "Database {0:s} doesn't exist.".format(fname)

        # Load mapping from gene/region identifiers to index values used in stored in inverted database.
        index_fname = InvertedRankingDatabase._derive_identifiers_fname(fname)
        assert os.path.isfile(fname), "Database index {0:s} doesn't exist.".format(index_fname)
        self.identifier2idx = self._load_identifier2idx(index_fname)

        # Load dataframe into memory: df = (n_features, max_rank).
        self.df = FeatherReader(fname).read().set_index(INDEX_NAME)
        self.max_rank = len(self.df.columns)

    def _load_identifier2idx(self, fname):
        with open(fname, 'r') as f:
            return {line.strip(): idx for idx, line in enumerate(f)}

    @property
    def total_genes(self) -> int:
        return len(self.identifier2idx)

    @property
    @memoize
    def genes(self) -> Tuple[str]:
        # noinspection PyTypeChecker
        return tuple(self.identifier2idx.keys())

    @property
    def region_identifiers(self) -> Tuple[str]:
        return self.genes

    def is_valid_rank_threshold(self, rank_threshold:int) -> bool:
        return rank_threshold <= self.max_rank

    def load_full(self) -> pd.DataFrame:
        # Loading the whole database into memory is not possible with an inverted database.
        # Decoration with a MemoryDecorator is not possible and will be prevented by raising
        # an exception.
        raise NotImplemented

    def load(self, gs: Type[GeneSignature]) -> pd.DataFrame:
        reference_identifiers = np.array([self.identifier2idx[identifier] for identifier in gs.genes], dtype=INVERTED_DB_DTYPE)
        ranked_identifiers = self.df.values
        return pd.DataFrame(data=build_rankings(ranked_identifiers, reference_identifiers),
                            index=self.df.index,
                            columns=gs.genes)

@njit(signature_or_function=uint32(uint32[:], uint32, uint32))
def find(ranked_identifiers, identifier, default_value):
    for idx in range(len(ranked_identifiers)):
        if ranked_identifiers[idx] == identifier:
            return idx
    return default_value

@njit(signature_or_function=uint32[:, :](uint32[:, :], uint32[:]), parallel=True)
def build_rankings(ranked_identifiers, reference_identifiers):
    rank_unknown = np.iinfo(INVERTED_DB_DTYPE).max
    n_features = ranked_identifiers.shape[0]; n_identifiers = len(reference_identifiers)
    result = np.empty(shape=(n_features, n_identifiers), dtype=INVERTED_DB_DTYPE)
    for row_idx in prange(n_features):
        for col_idx in range(n_identifiers):
            result[row_idx, col_idx] = find(ranked_identifiers[row_idx, :], reference_identifiers[col_idx], rank_unknown)
    return result


#@njit(signature_or_function=uint32[:, :](uint32[:, :], uint32[:]), parallel=True)
def build_rankings_lossy(ranked_identifiers, reference_identifiers):
    rank_unknown = np.iinfo(INVERTED_DB_DTYPE).max
    n_features = ranked_identifiers.shape[0]; n_identifiers = len(reference_identifiers)
    result = np.empty(shape=(n_features, n_identifiers), dtype=INVERTED_DB_DTYPE)
    for row_idx in prange(n_features):
        np.nonzero(np.where())
        for col_idx in range(n_identifiers):
            result[row_idx, col_idx] = find(ranked_identifiers[row_idx, :], reference_identifiers[col_idx], rank_unknown)
    return result

In [10]:
!sort -R {DB_FOLDER}/hg19-regions-220330-9species.inverted.identifiers.txt | head -10

chr11-reg88574
chr1-reg104773
chr16-reg64694
chr21-reg584
chr2-reg67061
chr19-reg10155
chr19-reg1136
chr15-reg26175
chr22-reg22992
chr3-reg37765
sort: Broken pipe


In [15]:
from random import shuffle
with open(os.path.join(DB_FOLDER, "hg19-regions-220330-9species.inverted.identifiers.txt") , 'r') as f:
    ids = list(map(lambda s: s.strip(), f))
    shuffle(ids)
    gs = GeneSignature("test_regions", "regionIDs", ids[:2500])

In [13]:
inv_db = InvertedRankingDatabase(os.path.join(DB_FOLDER, "hg19-regions-220330-9species.inverted.feather"), "test", "regionIDs")

In [14]:
len(inv_db.genes)

220330

In [16]:
inv_db.df.head()

,0,1,2,3,4,5,6,7,8,9,...,49990,49991,49992,49993,49994,49995,49996,49997,49998,49999
features,,,,,,,,,,,,,,,,,,,,,
elemento-AAAATGGCG,216747,146792,127978,146367,1799,192260,84271,36783,43385,45361,...,198785,81035,114643,54462,11823,14044,53253,173619,80486,95376
elemento-AAATCAAT,25546,207300,188600,128200,183402,58980,90776,63226,99184,65424,...,55706,194185,41845,144364,39523,74940,5901,192688,124017,88340
elemento-AAATGCAAA,93673,10046,29016,25672,188867,43400,130170,209852,44976,174826,...,93845,16801,199035,40073,328,42467,69899,167043,17119,138352
elemento-AAATTGCA,174175,88226,116446,31922,20921,116295,25672,216763,218405,21902,...,201940,218110,168422,219315,98532,104616,19057,210794,163261,61331
elemento-AACAGCTG,118437,116094,116772,41283,145391,206996,63244,74784,164603,31768,...,61705,135121,33140,66519,142502,135491,2230,214319,155278,115349


In [17]:
reference_identifiers = np.array([inv_db.identifier2idx[identifier] for identifier in gs.genes], dtype=INVERTED_DB_DTYPE)
ranked_identifiers = inv_db.df.values

In [18]:
reference_identifiers

array([186955,  73010, 134910, ...,    123,  40810, 203223], dtype=uint32)

In [20]:
ranked_identifiers[0, :]

array([216747, 146792, 127978, ..., 173619,  80486,  95376], dtype=uint32)

In [33]:
idx

array([   99,   104,   206,   238,   312,   350,   496,   692,   838,
        1149,  1548,  1609,  1721,  1763,  1863,  1868,  1889,  1967,
        1973,  2046,  2094,  2102,  2163,  2342,  2746,  2777,  2907,
        3010,  3165,  3195,  3305,  3453,  3549,  3636,  4070,  4226,
        4293,  4492,  4609,  4620,  4667,  5028,  5236,  5322,  5515,
        5535,  5595,  5719,  5789,  5855,  5860,  6001,  6029,  6067,
        6090,  6548,  6570,  6686,  6721,  6780,  6922,  6933,  6934,
        6940,  7109,  7122,  7192,  7283,  7321,  7440,  7473,  7504,
        7511,  7605,  7682,  7875,  7878,  7897,  7917,  7934,  7950,
        8173,  8259,  8279,  8371,  8451,  8566,  8682,  9022,  9139,
        9210,  9415,  9450,  9521,  9522,  9544,  9615,  9719,  9998,
       10196, 10230, 10249, 10266, 10322, 10431, 10550, 10641, 11003,
       11033, 11256, 11501, 11545, 11570, 11659, 11708, 11711, 11744,
       11815, 11906, 11917, 11967, 12115, 12233, 12333, 12356, 12366,
       12390, 12395,

In [ ]:
df = pd.DataFrame()

In [46]:
reference_identifiers = np.array([inv_db.identifier2idx[identifier] for identifier in gs.genes], dtype=INVERTED_DB_DTYPE)
ranked_identifiers = inv_db.df.values

In [47]:
ranked_identifiers.shape

(9713, 50000)

In [53]:
from tqdm import tqdm

In [61]:
def gen_series(ranked_identifiers, reference_identifiers):
    idx = np.nonzero(np.isin(ranked_identifiers, reference_identifiers))[0]
    return pd.Series(index=ranked_identifiers[idx], data=idx, dtype=INVERTED_DB_DTYPE)

In [68]:
rank_unknown = np.iinfo(INVERTED_DB_DTYPE).max
df_ = pd.concat([gen_series(ranked_identifiers[idx, :], reference_identifiers) for idx in tqdm(range(ranked_identifiers.shape[0]))], axis=1).T.fillna(rank_unknown).astype(INVERTED_DB_DTYPE)

100%|██████████| 9713/9713 [01:04<00:00, 149.75it/s]


In [72]:
idx2identifier = {idx:id for id, idx in inv_db.identifier2idx.items()}
df_.rename(columns=idx2identifier, inplace=False)

,chr1-reg101049,chr1-reg101094,chr1-reg101113,chr1-reg101350,chr1-reg101356,chr1-reg101502,chr1-reg10287,chr1-reg102870,chr1-reg103484,chr1-reg104796,...,chrX-reg56448,chrX-reg57117,chrX-reg57390,chrX-reg57420,chrX-reg57489,chrX-reg57787,chrX-reg57822,chrX-reg57988,chrY-reg2299,chrY-reg2562
0,4294967295,4294967295,4294967295,4294967295,4294967295,27332,4294967295,4294967295,4294967295,4294967295,...,4294967295,4294967295,7473,4294967295,4294967295,4294967295,4294967295,4294967295,32816,4294967295
1,4294967295,4294967295,4294967295,38963,4294967295,4294967295,4294967295,13772,4294967295,4294967295,...,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295
2,4294967295,24829,4294967295,14490,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,...,4294967295,4294967295,4294967295,4294967295,4294967295,35680,4294967295,4294967295,4294967295,4294967295
3,4294967295,4294967295,4294967295,37983,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,...,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295
4,4294967295,45222,4294967295,15055,4294967295,11472,4294967295,4294967295,4294967295,4294967295,...,4294967295,4294967295,4294967295,4294967295,37995,4294967295,4294967295,4294967295,19763,4294967295
5,4294967295,20185,4294967295,4294967295,4294967295,6553,4294967295,28140,4294967295,4294967295,...,4294967295,4294967295,40794,4294967295,4294967295,4294967295,4294967295,4294967295,34000,4294967295
6,4294967295,4294967295,4294967295,37039,4294967295,10942,4294967295,4294967295,4294967295,4294967295,...,4294967295,4294967295,47635,4294967295,4294967295,42841,4294967295,4294967295,22381,4294967295
7,4294967295,40842,4294967295,42847,4294967295,20454,4294967295,4294967295,4294967295,4294967295,...,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,15627,4294967295
8,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,...,4294967295,4294967295,17331,4294967295,4294967295,25933,4294967295,4294967295,4294967295,4294967295
9,4294967295,4294967295,22360,4294967295,4294967295,34294,4294967295,4294967295,4294967295,4294967295,...,11073,4294967295,11678,4294967295,12636,4294967295,4294967295,4294967295,4294967295,4294967295


In [69]:
df_

,123,131,136,158,162,177,407,408,555,684,...,219285,219545,219602,219619,219663,219742,219757,219786,220248,220273
0,4294967295,4294967295,4294967295,4294967295,4294967295,27332,4294967295,4294967295,4294967295,4294967295,...,4294967295,4294967295,7473,4294967295,4294967295,4294967295,4294967295,4294967295,32816,4294967295
1,4294967295,4294967295,4294967295,38963,4294967295,4294967295,4294967295,13772,4294967295,4294967295,...,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295
2,4294967295,24829,4294967295,14490,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,...,4294967295,4294967295,4294967295,4294967295,4294967295,35680,4294967295,4294967295,4294967295,4294967295
3,4294967295,4294967295,4294967295,37983,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,...,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295
4,4294967295,45222,4294967295,15055,4294967295,11472,4294967295,4294967295,4294967295,4294967295,...,4294967295,4294967295,4294967295,4294967295,37995,4294967295,4294967295,4294967295,19763,4294967295
5,4294967295,20185,4294967295,4294967295,4294967295,6553,4294967295,28140,4294967295,4294967295,...,4294967295,4294967295,40794,4294967295,4294967295,4294967295,4294967295,4294967295,34000,4294967295
6,4294967295,4294967295,4294967295,37039,4294967295,10942,4294967295,4294967295,4294967295,4294967295,...,4294967295,4294967295,47635,4294967295,4294967295,42841,4294967295,4294967295,22381,4294967295
7,4294967295,40842,4294967295,42847,4294967295,20454,4294967295,4294967295,4294967295,4294967295,...,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,15627,4294967295
8,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,4294967295,...,4294967295,4294967295,17331,4294967295,4294967295,25933,4294967295,4294967295,4294967295,4294967295
9,4294967295,4294967295,22360,4294967295,4294967295,34294,4294967295,4294967295,4294967295,4294967295,...,11073,4294967295,11678,4294967295,12636,4294967295,4294967295,4294967295,4294967295,4294967295


In [ ]:
inv_db.load(gs)

In [34]:
nr_chars = max(map(len,df_.columns))
nr_chars

14

In [39]:
import numpy as np
import pandas as pd
df_ = df.head()
nr_chars = max(map(len,df_.columns))
TOP_N = 50000
n_features = len(df_)
inverted_data = np.empty(shape=(n_features, TOP_N), dtype='U{}'.format(nr_chars))
for idx, (_, row) in enumerate(df_.iterrows()):
    inverted_data[idx, :] = row.sort_values(ascending=True).head(TOP_N).index
inverted_df = pd.DataFrame(data=inverted_data, index=df_.index, columns=list(range(TOP_N)))

In [40]:
inverted_df

,0,1,2,3,4,5,6,7,8,9,...,49990,49991,49992,49993,49994,49995,49996,49997,49998,49999
features,,,,,,,,,,,,,,,,,,,,,
elemento-AAAATGGCG,chrX-reg32698,chr3-reg11533,chr2-reg79638,chr3-reg110064,chr1-reg108107,chr7-reg58012,chr16-reg9028,chr11-reg40102,chr11-reg8174,chr11-reg9500,...,chr8-reg23479,chr16-reg45922,chr19-reg8404,chr12-reg77896,chr1-reg18112,chr1-reg28342,chr12-reg7113,chr5-reg8302,chr16-reg44441,chr17-reg70760
elemento-AAATCAAT,chr10-reg33634,chr9-reg32450,chr7-reg19391,chr2-reg80602,chr6-reg57419,chr13-reg43061,chr17-reg42372,chr14-reg3151,chr18-reg35521,chr14-reg50228,...,chr12-reg86983,chr7-reg70627,chr11-reg66224,chr22-reg4498,chr11-reg4974,chr15-reg9512,chr1-reg131822,chr7-reg62196,chr2-reg42740,chr17-reg33927
elemento-AAATGCAAA,chr17-reg60295,chr1-reg165040,chr10-reg66722,chr10-reg34533,chr7-reg2132,chr11-reg81814,chr20-reg16942,chr9-reg57882,chr11-reg91095,chr5-reg91502,...,chr17-reg635,chr1-reg42975,chr8-reg27484,chr11-reg52580,chr1-reg102464,chr11-reg73235,chr15-reg26297,chr5-reg112328,chr1-reg44410,chr21-reg22019
elemento-AAATTGCA,chr5-reg85184,chr17-reg33458,chr2-reg118522,chr10-reg94934,chr1-reg78218,chr2-reg116500,chr10-reg34533,chrX-reg32938,chrX-reg49548,chr1-reg89950,...,chr8-reg62041,chrX-reg47193,chr5-reg20272,chrX-reg5653,chr18-reg2385,chr19-reg22852,chr1-reg56518,chr9-reg66776,chr4-reg61821,chr14-reg15767
elemento-AACAGCTG,chr2-reg144559,chr2-reg114576,chr2-reg122253,chr11-reg61192,chr3-reg100749,chr9-reg28130,chr14-reg31649,chr15-reg7899,chr4-reg80579,chr10-reg9206,...,chr14-reg18607,chr20-reg48928,chr11-reg16370,chr14-reg59120,chr22-reg25707,chr20-reg5231,chr1-reg109841,chrX-reg11221,chr3-reg75807,chr2-reg103367


In [41]:
inverted_df.memory_usage(deep=True).sum()

17612575